In [80]:
import os 

import tensorflow as tf
import numpy as np
from PIL import Image

In [81]:
model_version = '7seg2912'

MODEL_FILE_PATH = f'../models/{model_version}.keras'
TFLITE_FILE_PATH = f'../models/{model_version}.tflite'
TFLITE_Q_FILE_PATH = f'../models/{model_version}q.tflite'

In [82]:
def yield_image() -> tuple:
    digits_dir = '../training/digits_resized'
    
    for image_path in os.listdir(digits_dir):
        image_in = Image.open(os.path.join(digits_dir, image_path))
        test_image = np.array(image_in, dtype="float32")
        img = np.reshape(test_image, [1, 32, 20, 3])
        
        correct_digit = int(image_path.split('_')[0])
        
        yield correct_digit, img

# Standard model

In [67]:
model = tf.keras.models.load_model(MODEL_FILE_PATH)

In [68]:
correct_res = []

for correct_digit, img in yield_image():
    pred_digit = np.argmax(model.predict(img, verbose=0))
    correct_res.append(int(pred_digit) == int(correct_digit))

In [69]:
std_arr = np.array(correct_res)

In [70]:
len(std_arr[std_arr == True]) / len(std_arr)

0.9983498349834984

# TFLite model

In [83]:
# Load the TFLite model in TFLite Interpreter
interpreter = tf.lite.Interpreter(TFLITE_FILE_PATH)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [84]:
tf_lite_res = []

for correct_digit, img in yield_image():
    interpreter.set_tensor(input_details[0]['index'], img)
    tflite_times = interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    pred_digit = output_data.argmax(axis=-1)
    
    tf_lite_res.append(int(pred_digit) == int(correct_digit))

C:\Users\sebas\AppData\Local\Temp\ipykernel_37400\1032507662.py:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  tf_lite_res.append(int(pred_digit) == int(correct_digit))


In [85]:
tflite_arr = np.array(tf_lite_res)

In [86]:
len(tflite_arr[tflite_arr == True]) / len(tflite_arr)

0.999211356466877

# TLITE quantized model

In [87]:
# Load the TFLite model in TFLite Interpreter
interpreter = tf.lite.Interpreter(TFLITE_Q_FILE_PATH)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [88]:
tf_lite_q_res = []

for correct_digit, img in yield_image():
    interpreter.set_tensor(input_details[0]['index'], img)
    tflite_times = interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    pred_digit = output_data.argmax(axis=-1)
    
    tf_lite_q_res.append(int(pred_digit) == int(correct_digit))

C:\Users\sebas\AppData\Local\Temp\ipykernel_37400\4026503442.py:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  tf_lite_q_res.append(int(pred_digit) == int(correct_digit))


In [89]:
tflite_q_arr = np.array(tf_lite_q_res)

In [90]:
len(tflite_arr[tflite_arr == True]) / len(tflite_arr)

0.999211356466877

In [91]:
len(tflite_arr[tflite_arr == False])

7